In [ ]:
import os

from sa_convlstm import SAConvLSTM
from convlstm import ConvLSTM
from utils import *

import torch
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader

import sys
import pickle
from tqdm import tqdm
import numpy as np
import math
import argparse
import json

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [ ]:
save_dir = './save_models/saclstm_pcrps_epoch_50_pinchu_pandamonium/'

In [ ]:
parser = argparse.ArgumentParser()
args = parser.parse_args(args="")

In [ ]:
with open(save_dir + 'args.txt', 'r') as f:
    args.__dict__ = json.load(f)

In [ ]:
testFolder = wb_dataset(root=args.data, dataset_type="test", frames_input=args.input_length,
                              frames_output=args.output_length, prob = args.prob_crps)

testLoader = torch.utils.data.DataLoader(testFolder,
                                          batch_size=args.batch_size,
                                          shuffle=False)

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

if args.convlstm and not args.prob_crps:
    network = ConvLSTM(args.input_dim, args.hidden_dim, args.output_dim,
                             args.kernel_size, device, dropout=args.dropout).to(device)
elif args.convlstm and args.prob_crps:
    network = ConvLSTM(2*args.input_dim, args.hidden_dim, 2*args.output_dim,
                       args.kernel_size, device, dropout=args.dropout).to(device)
elif args.saconvlstm and not args.prob_crps:   
    network = SAConvLSTM(args.input_dim, args.hidden_dim, args.output_dim, args.attn_dim,
                         args.kernel_size, device, dropout=args.dropout).to(device)
else:
    network = SAConvLSTM(2*args.input_dim, args.hidden_dim, 2*args.output_dim, args.attn_dim,
                         args.kernel_size, device, dropout=args.dropout).to(device)

optimizer = torch.optim.Adam(network.parameters(), lr=args.learn_rate, weight_decay=args.weight_decay)    
lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=0, verbose=True, min_lr=0.0001)

In [ ]:
max_ep = 0
for f in os.listdir(save_dir):
    split = f.split("_")
    if len(split)==2 and split[1] == "checkpoint.chk":
        if max_ep<int(split[0]): max_ep=int(split[0])
chkpnt = str(max_ep) + "_checkpoint.chk"                                         

In [ ]:
chk = torch.load(save_dir + chkpnt)
network.load_state_dict(chk['net'])

In [ ]:
loss_mc_test = infer(testLoader, args.input_length, network, args.output_dim, args.prob_crps)
print("test rmse: ", loss_mc_test)

In [ ]:
loss_WB_test = infer_WB(testFolder, testLoader, args.input_length, network, args.output_dim, args.prob_crps)
print("test lat rmse: ")
print(loss_WB_test)

In [ ]:
item = testFolder.__getitem__(367)

In [ ]:
output = network(torch.from_numpy(item[None, :7, ...]).float().to(device), train=False).detach().cpu().numpy()

In [ ]:
import numpy as np
from matplotlib.colors import Normalize
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point

In [ ]:
longs = np.arange(0, 360, 5.625)
lats = np.linspace(-90, 90, 32)

In [ ]:
item = item[:, :5, ...]*testFolder.long_std+testFolder.long_mean

In [ ]:
var = output[0][:, 5:, ...]*testFolder.long_std
output = output[0][:, :5, ...]*testFolder.long_std+testFolder.long_mean

In [ ]:
var = var**2

In [ ]:
channel=0

In [ ]:
np.arange(0, 360, 5.625).shape

In [ ]:
labels = item[7][channel]
wrap_data, wrap_lon = add_cyclic_point(labels, coord=longs, axis=1)
plt.figure(figsize=(20,9))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
im = ax.contourf(wrap_lon, lats, wrap_data, 100, transform=ccrs.PlateCarree(), norm = Normalize(vmin=240, vmax=305))
ax.set_global()
# cbar = ax.figure.colorbar(im, aspect=10, fraction=0.2, orientation='vertical')
# cbar.ax.tick_params(labelsize=40)
plt.show()

In [ ]:
labels = item[11][channel]
wrap_data, wrap_lon = add_cyclic_point(labels, coord=longs, axis=1)
plt.figure(figsize=(20,9))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.contourf(wrap_lon, lats, wrap_data, 100, transform=ccrs.PlateCarree(), norm = Normalize(vmin=240, vmax=305))
ax.set_global()
plt.show()

In [ ]:
# ---------------------

In [ ]:
labels = output[4][channel]
wrap_data, wrap_lon = add_cyclic_point(labels, coord=longs, axis=1)
plt.figure(figsize=(20,9))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
im = ax.contourf(wrap_lon, lats, wrap_data, 100, transform=ccrs.PlateCarree(), norm = Normalize(vmin=240, vmax=305))
ax.set_global()
cbar = ax.figure.colorbar(im, aspect=10, fraction=0.2, orientation='vertical')
cbar.ax.tick_params(labelsize=40)
plt.show()

In [ ]:
labels = var[4][channel]
wrap_data, wrap_lon = add_cyclic_point(labels, coord=longs, axis=1)
plt.figure(figsize=(20,9))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
im = ax.contourf(wrap_lon, lats, wrap_data, 100, transform=ccrs.PlateCarree(), norm = Normalize(vmin=0, vmax=35), cmap='Reds')
ax.set_global()
cbar = ax.figure.colorbar(im, aspect=10, fraction=0.2, orientation='vertical')
cbar.ax.tick_params(labelsize=40)
plt.show()

In [ ]:
# import scipy.ndimage
day = 4
labels = np.abs(item[day+7][channel] - output[day+0][channel])
wrap_data, wrap_lon = add_cyclic_point(labels, coord=longs, axis=1)
plt.figure(figsize=(20,9))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
# wrap_data=scipy.ndimage.filters.gaussian_filter(wrap_data, 0.4)
im = ax.contourf(wrap_lon, lats, wrap_data, 100, transform=ccrs.PlateCarree(), norm = Normalize(vmin=0, vmax=30), cmap='Reds')
ax.set_global()
# cbar = ax.figure.colorbar(im, aspect=10, fraction=0.2, orientation='vertical')
# cbar.ax.tick_params(labelsize=40)
plt.show()